In [1]:
import os
import sys

sys.path.insert(0, os.path.join("..", "..", ".."))
from well_logs import WellDataset
from well_logs.batchflow import Pipeline

In [2]:
DATASET_PATH = "/home/alexander/Documents/Data/vingapur_dataset/*"
well_ds = WellDataset(path=DATASET_PATH, dirs=True, sort=True)

In [3]:
MATCHED_DATASET_PATH = "/home/alexander/Documents/Data/matched_vingapur_dataset/"

In [4]:
matching_modes = [
    "GK ~ core_logs.GK",
    "DENSITY ~ core_logs.DENSITY",
    "DENSITY ~ core_properties.DENSITY",
    "DENSITY ~ core_properties.POROSITY",
    "DT ~ core_properties.POROSITY",
    "NKTD ~ core_properties.POROSITY",
]

In [5]:
matching_pipeline = (Pipeline()
    .check_regularity()
    .match_core_logs(mode=matching_modes, save_report=True)
    .dump(MATCHED_DATASET_PATH)
    .run(batch_size=1, n_epochs=1, shuffle=False, drop_last=False, bar=True, lazy=True)
)

In [6]:
(well_ds >> matching_pipeline).run()

 34%|███▍      | 50/147 [29:47<43:04, 26.65s/it]   
